In [6]:
# !kaggle datasets download -d eswarreddy12/family-guy-dialogues-with-various-lexicon-ratings
# !mkdir -p data
# !unzip family-guy-dialogues-with-various-lexicon-ratings.zip -d data
# !rm family-guy-dialogues-with-various-lexicon-ratings.zip

In [7]:
!ls data/

 Family_Guy_Final_NRC_AFINN_BING.csv   GPT_lex11.csv   Ratings_FG5.csv


In [8]:
import csv

file_path = "data/Family_Guy_Final_NRC_AFINN_BING.csv"

# Print headings
with open(file_path, "r") as f:
    csvf = csv.reader(f, delimiter=",", quotechar="'")
    headings = next(csvf)  # Read the first row (headings)
    print(", ".join(headings))

Season, Episode, Time_Stamp, Dialogue, NRC_Sentiment, AFINN_Sentiment, AFINN_Sentiment_Score, BING_Sentiment


In [9]:
corpus = []

# Read dialogue and create the corpus
with open(file_path, "r") as f:
    csvf = csv.DictReader(f, delimiter=",", quotechar="'")
    for row in csvf:
        dialogue = row["Dialogue"].strip('"')  # Remove double quotes
        corpus.append(dialogue)

In [10]:
# Print the first 5 entries of the corpus
print("Corpus:")
for i in range(5):
    print(corpus[i])

print(f"Corpus line num: {len(corpus)}")

corpus_str = "\n".join(corpus)

print(f"Corpus str len: {len(corpus_str)}")

print(corpus_str[:1000])

Corpus:
Mom
Greg
No
He's lying. There's no doubt about that.
Greg
Corpus line num: 154844
Corpus str len: 3623337
Mom
Greg
No
He's lying. There's no doubt about that.
Greg
That'll give you time to think about what you did.
Man!
That'll teach him.
Jan
Smoking. How does a boy like that go so wrong?
They live in a crummy neighborhood.
The Bradys?
Yeah. They got robbers
You folks want some pancakes?
No
Mom
Meg
You know
Excellent! The mind-control device is nearing completion!
Stewie
Damn you
You've impeded my work since the day I escaped from your wretched womb.
Don't pout
But
No toys
Very well
Mark my words
Mom
Don't touch the thermostat
Come on. This thing goes up to 90.
Who touched the thermostat?
God
Brain implant
Tells you when the kids mess with the dial.
My thing went off! Your thermostat okay?
Yeah
Is my kid over here?
Forget it! False alarm!
Ass ahoy.
Peter
He's going to a stag party.
Lois
I am the man of the house.
As the man
Look
Come on. You're worrying about nothing.
Remember 

In [11]:
# Unique chars in the corpus

chars = sorted(list(set(corpus_str)))
vocab_size = len(chars)
print("".join(chars))
print(f"Vocab size: {vocab_size}")

# TODO: would need to remove chars from corpus too
# chars = chars[:91] # remove special chars
# vocab_size = len(chars)
# print(''.join(chars))
# print(f"Vocab size: {vocab_size}")


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{}¤ª³¶¿ÁÂÃÇÉÊÓàáâãçèéêíñóôúû‰™
Vocab size: 122


In [12]:
# Create mapping from chars to ints

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

print(encode("Hi there!"))
print(decode(encode("Hi there!")))

[41, 73, 1, 84, 72, 69, 82, 69, 2]
Hi there!


In [13]:
import torch

data = torch.tensor(encode(corpus_str), dtype=torch.long)

In [14]:
print(data.shape)
print(data.dtype)
print(data[:100])

torch.Size([3623337])
torch.int64
tensor([46, 79, 77,  0, 40, 82, 69, 71,  0, 47, 79,  0, 41, 69,  8, 83,  1, 76,
        89, 73, 78, 71, 15,  1, 53, 72, 69, 82, 69,  8, 83,  1, 78, 79,  1, 68,
        79, 85, 66, 84,  1, 65, 66, 79, 85, 84,  1, 84, 72, 65, 84, 15,  0, 40,
        82, 69, 71,  0, 53, 72, 65, 84,  8, 76, 76,  1, 71, 73, 86, 69,  1, 89,
        79, 85,  1, 84, 73, 77, 69,  1, 84, 79,  1, 84, 72, 73, 78, 75,  1, 65,
        66, 79, 85, 84,  1, 87, 72, 65, 84,  1])


In [15]:
# Split dataset into train and validation
n = int(0.9 * len(corpus_str))

train_data, val_data = data[:n], data[n:]

In [16]:
block_size = 8
train_data[: block_size + 1]

tensor([46, 79, 77,  0, 40, 82, 69, 71,  0])

In [17]:
x = train_data[:block_size]
y = train_data[1 : block_size + 1]

for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([46]) the target is 79
when input is tensor([46, 79]) the target is 77
when input is tensor([46, 79, 77]) the target is 0
when input is tensor([46, 79, 77,  0]) the target is 40
when input is tensor([46, 79, 77,  0, 40]) the target is 82
when input is tensor([46, 79, 77,  0, 40, 82]) the target is 69
when input is tensor([46, 79, 77,  0, 40, 82, 69]) the target is 71
when input is tensor([46, 79, 77,  0, 40, 82, 69, 71]) the target is 0


In [18]:
torch.manual_seed(1337)

batch_size = 4  # how many independent sequences will we process in parallel?
block_size = 8  # what is the max context length for predictions?


def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data

    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])

    return x, y


(
    xb,
    yb,
) = get_batch("train")

print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("\n-----\n")

for b in range(batch_size):  # batch dim
    for t in range(block_size):  # time dim
        context = xb[b, : t + 1]
        target = yb[b, 1]
        print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[68,  1, 69, 65, 71, 76, 69, 83],
        [ 1, 70, 65, 82, 84,  1, 83, 79],
        [ 1, 84, 72, 69,  1, 52, 65, 66],
        [83,  1, 82, 73, 68, 73, 67, 85]])
targets:
torch.Size([4, 8])
tensor([[ 1, 69, 65, 71, 76, 69, 83,  1],
        [70, 65, 82, 84,  1, 83, 79, 85],
        [84, 72, 69,  1, 52, 65, 66, 66],
        [ 1, 82, 73, 68, 73, 67, 85, 76]])

-----

when input is [68] the target is 69
when input is [68, 1] the target is 69
when input is [68, 1, 69] the target is 69
when input is [68, 1, 69, 65] the target is 69
when input is [68, 1, 69, 65, 71] the target is 69
when input is [68, 1, 69, 65, 71, 76] the target is 69
when input is [68, 1, 69, 65, 71, 76, 69] the target is 69
when input is [68, 1, 69, 65, 71, 76, 69, 83] the target is 69
when input is [1] the target is 65
when input is [1, 70] the target is 65
when input is [1, 70, 65] the target is 65
when input is [1, 70, 65, 82] the target is 65
when input is [1, 70, 65, 82, 84] the targ

In [19]:
import torch.nn as nn
from torch.nn import functional as F


class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors of ints
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)  # (B, C, T)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context

        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # ==> (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx

In [20]:
import math

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print("Expected loss: " + str(-math.log(1 / vocab_size)))

print("\n-----\n")

idx = torch.zeros((1, 1), dtype=torch.long)

generated = decode(m.generate(idx , max_new_tokens=100)[0].tolist())

print(generated)

torch.Size([32, 122])
tensor(5.2793, grad_fn=<NllLossBackward0>)
Expected loss: 4.804021044733257

-----


Qiíñs\‰"=eÉ`á*jè:]ñ[¤[,Bàuªed?ª/IAX.q[ãhô4Sk<`TAI5ÁnéKkÊYvu>1XSÁXStú‰}4<Ç=zi1tÇW+SÂHÃH}}Yñw/j#>I>w


In [23]:
# Train model
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [29]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # eval the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.41341495513916


In [30]:
generated = decode(m.generate(idx , max_new_tokens=500)[0].tolist())
print(generated)


Fe b
I'tot ish.
Dan
AAl the sa thth rs..
H
I thy t.
Ne y ouy coovisugh
Noyt wesnay whawr f th
Dales tt GEte's thag ay 2 toll,120s sthe yom. fe.
Wewe y vetowon'tho mecthoundnd thlly fine Finer gost betalle y. llies t.
Goncowank.
I my
Whauat lidsn tha y ju- lonwerid cale oulan'towhtate oppacongsthr
Bou bery mus thedit ogheroingl
Noer, utthat
I tesuininofo tace as az
Oht
RAnd t pe
Hw tha y
An
Youpr arenelil
Noit d. op y
P.
Whous corourcet
Homasigout d!
Pere fof pasele's s k! o..
Thoit wir wo al yod
